Rayleigh-Taylor instability
======

<table><tr><td><img src='./pic/kaus2010_RT_image0000.png'></td></tr></table>
Initial model setup with buoyancy perturbation.

This notebook models the Rayleigh-Taylor instability outlined in van Kaus *et al.* (2010) for the FSSA algorithm test. 

The system of equations is given by

$$
    \nabla \cdot \left( \eta \nabla \dot\varepsilon \right) - \nabla p = -\rho g \mathbf{\hat z}
$$

$$
    \nabla \cdot \mathbf{v} = 0
$$

**Keywords:** particle swarms, Stokes system, advective diffusive systems

**References**

- Kaus, B.J.P., et al., A stabilization algorithm for geodynamic numerical simulations with a free surface. Phys.
Earth Planet. In. (2010), doi:10.1016/j.pepi.2010.04.007

In [1]:
import underworld as uw
from underworld import function as fn
import underworld.visualisation as vis
from underworld.scaling import units as u
from underworld.scaling import dimensionalise, non_dimensionalise
import math
import numpy as np
import os

In [2]:
# output
input_path  = 'input/05_Rayleigh_Taylor/'
output_path = 'output/'

model_end_step   = 100.
res              = 64
stokes_inner_tol = 1e-6
stokes_outer_tol = 1e-5

# search and build, if required, an output path
if uw.mpi.rank==0:
    try:
        if not os.path.exists("./"+output_path+"/"):
            os.makedirs("./"+output_path+'/')
    except:
        raise

In [3]:
# build reference units
KL_meters   = 500 * u.kilometer
K_viscosity = 1e20  * u.pascal * u.second
K_density   = 3200 * u.kilogram / u.meter**3

# compute dependent scaling units 
KM_kilograms = K_density * KL_meters**3
KT_seconds   = KM_kilograms / ( KL_meters * K_viscosity )
K_substance  = 1. * u.mole
#Kt_degrees   = 1. * u.kelvin

scaling_coefficients = uw.scaling.get_coefficients()
scaling_coefficients["[length]"]      = KL_meters.to_base_units()
#scaling_coefficients["[temperature]"] = Kt_degrees.to_base_units()
scaling_coefficients["[time]"]        = KT_seconds.to_base_units()
scaling_coefficients["[mass]"]        = KM_kilograms.to_base_units()

gravity   = non_dimensionalise( 9.81 * u.meter / u.second**2)


ymaxCoord = non_dimensionalise(0.* u.kilometer)
yminCoord = non_dimensionalise(-500* u.kilometer)
xmaxCoord = non_dimensionalise(250* u.kilometer)
xminCoord = non_dimensionalise(-250* u.kilometer)

In [4]:
# build mesh and mesh variables
resolution       = (64,64)
mesh = uw.mesh.FeMesh_Cartesian( elementType = 'Q1/dQ0', 
                                 elementRes  = resolution, 
                                 minCoord    = [xminCoord,yminCoord], 
                                 maxCoord    = [xmaxCoord,ymaxCoord],
                                 periodic    = [False, False] )

velocityField   = uw.mesh.MeshVariable( mesh=mesh,         nodeDofCount=mesh.dim )
pressureField   = uw.mesh.MeshVariable( mesh=mesh.subMesh, nodeDofCount=1 )

velocityField.data[:] = [0.,0.]
pressureField.data[:] = 0.

In [5]:
# Create a swarm.
swarm = uw.swarm.Swarm( mesh=mesh )

# Create a data variable. It will be used to store the material index of each particle.
materialIndex = swarm.add_variable( dataType="int", count=1 )

# Create a layout object, populate the swarm with particles.
swarmLayout = uw.swarm.layouts.PerCellSpaceFillerLayout( swarm=swarm, particlesPerCell=20 )
swarm.populate_using_layout( layout=swarmLayout )

In [6]:
# define these for convience. 
denseIndex = 0
lightIndex = 1

# material perturbation from van Keken et al. 1997
wavelength = non_dimensionalise(xmaxCoord*2)
amplitude  = non_dimensionalise(5*u.kilometer)
offset     = non_dimensionalise(-100.*u.kilometer)
k = 2. * math.pi / wavelength

# Create function to return particle's coordinate
coord = fn.coord()

# Define the material perturbation, a function of the x coordinate (accessed by `coord[0]`).
perturbationFn = offset + amplitude*fn.math.cos( k*coord[0] )

# Setup the conditions list. 
# If z is less than the perturbation, set to lightIndex.
conditions = [ ( perturbationFn > coord[1] , lightIndex ),
               (                      True , denseIndex ) ]

# The swarm is passed as an argument to the evaluation, providing evaluation on each particle.
# Results are written to the materialIndex swarm variable.
materialIndex.data[:] = fn.branching.conditional( conditions ).evaluate(swarm)


'output/image_0.png'

In [7]:
fig1 = vis.Figure() #title="Material", figsize=(400,400), quality=2, rulers=True)
fig1.Points(swarm, materialIndex, fn_size=2.,discret=True,colourBar = False) 
fig1.VectorArrows(mesh, velocityField) 
fig1.show()
fig1.save_image(output_path+"image_0")

In [8]:
light_density = non_dimensionalise(3200 * u.kilogram / u.meter**3)
dense_density = non_dimensionalise(3300 * u.kilogram / u.meter**3)
light_viscosity = non_dimensionalise(1e20 * u.pascal * u.second)
dense_viscosity = non_dimensionalise(1e21 * u.pascal * u.second)


# Set a density of '0.' for light material, '1.' for dense material.
densityMap   = { lightIndex:light_density, denseIndex:dense_density }
densityFn    = fn.branching.map( fn_key = materialIndex, mapping = densityMap )

# Set a viscosity value of '1.' for both materials.
viscosityMap = { lightIndex:light_viscosity, denseIndex:dense_viscosity }
fn_viscosity  = fn.branching.map( fn_key = materialIndex, mapping = viscosityMap )

# Define a vertical unit vector using a python tuple.
z_hat = ( 0.0, 1.0 )

# Create buoyancy force vector
buoyancyFn = -densityFn*z_hat

In [9]:
# Construct node sets using the mesh specialSets
iWalls = mesh.specialSets["Left_VertexSet"]   + mesh.specialSets["Right_VertexSet"]
jWalls = mesh.specialSets["Bottom_VertexSet"] + mesh.specialSets["Top_VertexSet"]
botWalls = mesh.specialSets["Bottom_VertexSet"]
allWalls = iWalls + jWalls

# for test FSSA
# Side boundaries are free slip, 
# the lower boundary is no-slip 
# and the upper boundary is a free surface
# stokesBC = uw.conditions.DirichletCondition( variable      = velocityField, 
#                                              indexSetsPerDof = (iWalls+botWalls, botWalls) )


# for test free slip top
stokesBC = uw.conditions.DirichletCondition( variable      = velocityField, 
                                             indexSetsPerDof = (iWalls+botWalls,jWalls) )


In [10]:
stokes = uw.systems.Stokes( velocityField = velocityField, 
                            pressureField = pressureField,
                            conditions    = stokesBC,
                            fn_viscosity  = fn_viscosity, 
                            fn_bodyforce  = buoyancyFn )

solver = uw.systems.Solver( stokes )

# Optional solver settings
if(uw.mpi.size==1):
    solver.set_inner_method("lu")
solver.set_inner_rtol(stokes_inner_tol) 
solver.set_outer_rtol(stokes_outer_tol) 

# Create a system to advect the swarm
advector = uw.systems.SwarmAdvector( swarm=swarm, velocityField=velocityField, order=2 )

In [11]:
# Initialise time and timestep.
time = 0.
step = 0

# parameters for output
outputEvery  = 10
timeVal     = []
vrmsVal     = []

# Save mesh and retain file handle for future xdmf creation
meshFileHandle = mesh.save(output_path+"Mesh.h5")

In [12]:
# define an update function
def update():
    dt = advector.get_max_dt() # retrieve the maximum possible timestep from the advection system.
    advector.integrate(dt)     # advect step.
    return time+dt, step+1

In [13]:
while step < model_end_step:

    # Get solution
    solver.solve()
    
    # Calculate the RMS velocity.
    vrms = stokes.velocity_rms()

    # Record values into arrays
    if(uw.mpi.rank==0):
        vrmsVal.append(vrms)
        timeVal.append(time)
    
    # Output to disk
    if step%outputEvery == 0:
        if(uw.mpi.rank==0):
            print('step = {0:6d}; time = {1:.3e}; v_rms = {2:.3e}'.format(step,time,vrms))

        filename = output_path+"/velocityField."+str(step)
        vFH      = velocityField.save(filename+".h5")
        velocityField.xdmf( filename, vFH, "velocity", meshFileHandle, "Mesh", time )
        
        filename = output_path+"/pressureField."+str(step)
        pFH      = pressureField.save(filename+".h5")
        pressureField.xdmf(filename, pFH, "pressure", meshFileHandle, "Mesh", time )
        
        outputFilename = output_path+"image"+str(step).zfill(4)
        fig1.save_image(outputFilename)

    # We are finished with current timestep, update.
    time, step = update()
    
if(uw.mpi.rank==0):
    print('step = {0:6d}; time = {1:.3e}; v_rms = {2:.3e}'.format(step,time,vrms))

step =      0; time = 0.000e+00; v_rms = 6.955e-07
step =     10; time = 1.690e+04; v_rms = 5.727e-06
step =     20; time = 2.113e+04; v_rms = 1.122e-05
step =     30; time = 2.357e+04; v_rms = 1.745e-05
step =     40; time = 2.528e+04; v_rms = 2.448e-05
step =     50; time = 2.659e+04; v_rms = 3.190e-05
step =     60; time = 2.766e+04; v_rms = 3.932e-05
step =     70; time = 2.858e+04; v_rms = 4.552e-05
step =     80; time = 2.943e+04; v_rms = 4.887e-05
step =     90; time = 3.027e+04; v_rms = 4.729e-05
step =    100; time = 3.119e+04; v_rms = 4.122e-05


In [14]:
viscosityRatio = dense_viscosity / light_viscosity


if uw.mpi.rank==0:
    
    maxIndex = np.argmax(vrmsVal)

    print('Viscosity ratio = {0:.2f}'.format(viscosityRatio))
    print('    t(max vrms) = {0:.2f}'.format(timeVal[maxIndex]))
    print('           vrms = {0:.4e}'.format(vrmsVal[maxIndex]))
    
    # output a summary file with benchmark values (useful for parallel runs)
    np.savetxt(output_path+'summary.txt', [viscosityRatio, timeVal[maxIndex], vrmsVal[maxIndex]])

Viscosity ratio = 10.00
    t(max vrms) = 29766.43
           vrms = 4.9101e-05


In [15]:
# if uw.mpi.size == 1:
#     if   viscosityRatio == 1.0 :
#         data = np.loadtxt(input_path+'VrmsCaseA.txt', unpack=True )
#     elif viscosityRatio == 0.1 :
#         data = np.loadtxt(input_path+'VrmsCaseB.txt', unpack=True )
#     elif viscosityRatio == 0.01 :
#         data = np.loadtxt(input_path+'VrmsCaseC.txt', unpack=True )
#     else :
#         print('No specific data found - default to Case A')
#         data = np.loadtxt(input_path+'VrmsCaseA.txt', unpack=True )

#     # Load into data arrays to compare with timevals and vrmsvals from above.
#     timeCompare, vrmsCompare = data[0], data[1] 
#     # These can be copied onto timevals and vrmsvals to skip running the time loop.    
#     uw.utils.matplotlib_inline()

#     import matplotlib.pyplot as pyplot
#     fig = pyplot.figure()
#     fig.set_size_inches(12, 6)
#     ax = fig.add_subplot(1,1,1)
#     ax.plot(timeCompare, vrmsCompare, color = 'black') 
#     ax.plot(timeVal, vrmsVal, color = 'red', marker=".", markersize=10) 
#     ax.set_xlabel('Time')
#     ax.set_ylabel('RMS velocity')
#     ax.set_xlim([0.0,1000.0])